# METADATA CONVERSION

Download dataset IMDB - faces only (7 GB) from https://data.vision.ee.ethz.ch/cvl/rrothe/imdb-wiki/
Copy the downloaded directory imdb_crop in the project folder dataset/, then run this notebook.

This notebook converts the imdb_crop images metadata from Matlab format into Pandas format, so they can be used with Python.

The ML models for classification must inherit from src/models/Model, the DataManager class reads the images and extracts the labels

In [1]:
import scipy.io as sio
import pandas as pd

In [2]:
# Matlab file path
METADATA_CROPPED_FILE_MATLAB = '../dataset/imdb_crop/imdb.mat'

In [6]:
mat_obj = sio.loadmat(METADATA_CROPPED_FILE_MATLAB)
mat_dta = mat_obj['imdb']
data_dict = {n: mat_dta[n][0, 0][0] for n in mat_dta.dtype.names}
# Extract celebrities names
celeb_names_dict = {k: v for k, v in enumerate(data_dict['celeb_names'], 1)}
# Remove key from dict
_ = data_dict.pop('celeb_names', None)

error: Error -3 while decompressing data: invalid code lengths set

In [7]:
df = pd.DataFrame.from_dict(data_dict)
df.head()

NameError: name 'data_dict' is not defined

In [8]:
from datetime import datetime, timedelta
matlab2datetime = lambda matlab_datenum: (datetime.fromordinal(int(matlab_datenum)) + timedelta(days=matlab_datenum%1)).year

# Extract full path
df['full_path'] = df['full_path'].apply(lambda x: x[0]).astype('str')
# Extract face location
df['face_location'] = df['face_location'].apply(lambda x: x[0])
# Extract face location
df['name'] = df['name'].apply(lambda x: x[0]).astype('str')
# Convert dab to standard datetime
df['standard_dob'] = df['dob'].apply(matlab2datetime).astype('uint16')
# Compute age
df['age'] = df.apply(lambda x:x['photo_taken']-x['standard_dob'], axis=1).astype('uint16')

NameError: name 'df' is not defined

In [9]:
# Reorder columns
columns = df.columns.tolist()
head = ['age', 'gender', 'full_path', 'name', 'face_location']
tail = set(columns) - set(head)
ordered_columns = head + list(tail)
# Ordered columns
df = df[ordered_columns]
df.head()

NameError: name 'df' is not defined

In [10]:
# Remove invalid rows
len_before = len(df)
print('Len before: ', len_before)
df = df.query('age<=100')
len_after = len(df)
print('Lean after: ', len_after)
print(f'Invalid rows: {(1 - len_after/len_before)*100:.3f}%')


NameError: name 'df' is not defined

In [14]:
from os import path

# Dump dataframe
head, tail = path.split(METADATA_CROPPED_FILE_MATLAB)

path_serialized = path.join(head, 'imdb.pickle')
df.to_pickle(path_serialized)

NameError: name 'df' is not defined

In [16]:
# Test read
df = pd.read_pickle(path_serialized)
df.head()

,age,gender,full_path,name,face_location,standard_dob,photo_taken,dob,face_score,second_face_score,celeb_id
0,68,1.0,01/nm0000001_rm124825600_1899-5-10_1968.jpg,Fred Astaire,"[1072.926, 161.838, 1214.7839999999999, 303.69...",1900,1968,693726,1.459693,1.118973,6488
1,70,1.0,01/nm0000001_rm3343756032_1899-5-10_1970.jpg,Fred Astaire,"[477.184, 100.352, 622.592, 245.76]",1900,1970,693726,2.543198,1.852008,6488
2,68,1.0,01/nm0000001_rm577153792_1899-5-10_1968.jpg,Fred Astaire,"[114.96964308962852, 114.96964308962852, 451.6...",1900,1968,693726,3.455579,2.985660,6488
3,68,1.0,01/nm0000001_rm946909184_1899-5-10_1968.jpg,Fred Astaire,"[622.8855056426588, 424.21750383700805, 844.33...",1900,1968,693726,1.872117,NaN,6488
4,68,1.0,01/nm0000001_rm980463616_1899-5-10_1968.jpg,Fred Astaire,"[1013.8590023603723, 233.8820422075853, 1201.5...",1900,1968,693726,1.158766,NaN,6488


In [17]:
df.describe()


,age,gender,standard_dob,photo_taken,dob,face_score,second_face_score,celeb_id
count,460025.000000,451571.000000,460025.000000,460025.000000,460025.000000,4.600250e+05,213541.000000,460025.000000
mean,35.935149,0.582041,1969.562580,2005.497729,719185.792540,-inf,2.453057,10117.013497
std,13.109242,0.493224,15.228522,8.993045,5561.522003,NaN,1.064391,5741.796404
min,0.000000,0.000000,1884.000000,1961.000000,687776.000000,-inf,0.730926,1.000000
25%,27.000000,0.000000,1962.000000,2004.000000,716370.000000,1.759469e+00,1.583918,5295.000000
50%,34.000000,1.000000,1972.000000,2008.000000,719928.000000,2.980731e+00,2.355290,10066.000000
75%,43.000000,1.000000,1980.000000,2011.000000,723066.000000,4.006804e+00,3.228120,14922.000000
max,100.000000,1.000000,2013.000000,2015.000000,734963.000000,7.381689e+00,6.395435,20284.000000
